In [1]:
from evdev import InputDevice, ecodes 
device = InputDevice('/dev/input/event0')  

#### The following code will give each button or axis a name

In [2]:
code2name = {}
for type_, code_ in device.capabilities(verbose=True, absinfo=False).items(): 
    if type_[0] in ['EV_KEY', 'EV_ABS']:
        for name, code in code_:  
            if isinstance(name,list): 
                if   'BTN_A' in name: code2name[code] = 'BTN_A'  
                elif 'BTN_B' in name: code2name[code] = 'BTN_B'   
                elif 'BTN_X' in name: code2name[code] = 'BTN_X'  
                elif 'BTN_Y' in name: code2name[code] = 'BTN_Y'  
                continue
            code2name[code] = name
print(code2name)

{158: 'KEY_BACK', 304: 'BTN_A', 305: 'BTN_B', 306: 'BTN_C', 307: 'BTN_X', 308: 'BTN_Y', 309: 'BTN_Z', 310: 'BTN_TL', 311: 'BTN_TR', 312: 'BTN_TL2', 313: 'BTN_TR2', 314: 'BTN_SELECT', 315: 'BTN_START', 316: 'BTN_MODE', 317: 'BTN_THUMBL', 318: 'BTN_THUMBR', 0: 'ABS_X', 1: 'ABS_Y', 2: 'ABS_Z', 5: 'ABS_RZ', 9: 'ABS_GAS', 10: 'ABS_BRAKE', 16: 'ABS_HAT0X', 17: 'ABS_HAT0Y'}


#### We need the code for every digital button since one message will be sent to server each time we press one

In [3]:
digital = ['KEY_BACK', 'BTN_A', 'BTN_B', 'BTN_C', 'BTN_X', 'BTN_Y', 'BTN_Z', 'BTN_TL', 'BTN_TR', 'BTN_TL2', 'BTN_TR2', 'BTN_SELECT', 'BTN_START', 'BTN_MODE', 'BTN_THUMBL', 'BTN_THUMBR', 'ABS_HAT0X', 'ABS_HAT0Y']
digital = [code for code, name in code2name.items() if name in digital] 

#### We also need the code for every analog axis but we will send the position of analog axes periodically since the position is changing quickly and constantly (we cannot send the change every time it is detected because that is expansive). 

In [4]:
analog_position = {
        'ABS_X':35000, # default value for left  stick X   
        'ABS_Y':32000, # default value for left  stick Y
        'ABS_Z':33000, # default value for right stick X
       'ABS_RZ':33000, # default value for right stick Y
      'ABS_GAS':0,     # default value for right trigger              
    'ABS_BRAKE':0,     # default value for left  trigger    
}
analog = [code for code, name in code2name.items() if name in analog_position.keys()] 

#### We will create a hook to log the position of analog axes and send a message once a button is pressed

In [5]:
import requests 

def hook(analog, analog_position, digital):
    for event in device.read_loop(): 
        if event.code in analog and event.type == ecodes.EV_ABS: 
            analog_position[code2name[event.code]] = event.value  # analog_position variable is changing quickly and constantly 
        elif event.code in digital and event.type in [ecodes.EV_ABS, ecodes.EV_KEY]:
            requests.post('http://192.168.0.222:9999/receiver', data={code2name[event.code]:event.value}) # send msg for each button pressing
            
import threading
hook_thread = threading.Thread(target=hook, args=(analog, analog_position, digital))
hook_thread.start() # this background thread will monitor axis event and modify variable axis in a real-time manner 

#### This thread would report the position of analog axes to server periodically

In [6]:
import time
def analog_reporter(analog_position, interval=0.5):
    while True: 
        requests.post('http://192.168.0.222:9999/receiver', data=analog_position) 
        time.sleep(interval)

import threading
reporter_thread = threading.Thread(target=analog_reporter, args=(analog_position, 2))
reporter_thread.start() # this background thread will report axis value every 2 seconds